# 비디오(Video) 질의 응답 LLM (Gemini)

**주요 흐름**

1. `File API`를 사용하여 비디오 파일을 업로드합니다.
2. `GenerateContent` 요청을 통해 비디오에 대한 질문을 요청합니다.
3. 생성된 응답을 확인합니다.
4. 업로드한 Video 파일을 삭제합니다.


**중요:** 

- 본 튜토리얼의 `File API` 는 인증 및 접근을 위해 `API keys`를 사용합니다. 

- 업로드된 파일은 `API key`의 클라우드 프로젝트와 연결됩니다. 

다른 `Gemini API` 와 달리, `API key`는 `File API`에 업로드한 데이터에 대한 접근 권한도 부여하므로 `API key`를 안전하게 보관하는 데 특별히 주의해야 합니다.

**Reference**

- [Gemini API(Cookbook) - Video](https://ai.google.dev/api/rest/v1/models/generateContent#media)

**API KEY 발급**

- [링크](https://makersuite.google.com/app/apikey?hl=ko) 에서 API KEY를 발급받아주세요.
- 사용자의 Google API 키를 환경 변수 `GOOGLE_API_KEY`로 설정합니다.

`.env` 파일에 아래와 같이 입력합니다.

```
GOOGLE_API_KEY=<사용자의 API KEY>
```


In [9]:
# LangSmith 추적 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("CH04-Gemini-Video")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Gemini-Video


## 비디오 업로드

`Gemini API` 는 비디오 파일 형식을 직접 수용합니다. 

**제한사항**

- `File API`는 2GB 이하의 파일을 수용하며 프로젝트당 최대 20GB의 파일을 저장할 수 있습니다. 
- 파일은 2일 동안 유지되며 API에서 **다운로드할 수 없습니다**. 

본 예제는 테디노트 YouTube 채널에 게시된 비디오를 사용합니다. (다른 비디오로 교체하여 진행해도 좋습니다)

- [🧑‍💻 #PDF 전처리 할 수 있는 건 다 해봤음.](https://youtu.be/O3qFWRObAXw)

In [ ]:
# 파일 다운로드 후 teddynote-sample-video.mp4 파일로 저장
!wget "https://www.dropbox.com/scl/fi/ugue14fyo010jgc7wuh4g/teddynote-sample-video.mp4?rlkey=wcsiktklt7jgoibsluft3m6z9&st=prv4p2uu&dl=1" -qO teddynote-sample-video.mp4

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


아래 비디오 파일의 경로를 입력합니다.

In [14]:
# 비디오 파일 이름 지정
video_file_name = "teddynote-sample-video.mp4"

다음으로는 `File API`를 사용하여 비디오 파일을 업로드합니다.

In [ ]:
!pip install -U google-generativeai

In [17]:
import google.generativeai as genai

# 파일 업로드 진행 메시지 출력
print(f"파일을 업로드 중입니다...")

# 파일 업로드 및 파일 객체 반환
video_file = genai.upload_file(path=video_file_name)

# 업로드 완료 메시지 및 파일 URI 출력
print(f"업로드 완료: {video_file.uri}")

파일을 업로드 중입니다...
업로드 완료: https://generativelanguage.googleapis.com/v1beta/files/lvj0w4tmclhv


파일을 업로드한 후, `files.get` 을 호출하여 API가 파일을 성공적으로 완료되었는지 확인할 수 있습니다. 

`files.get`은  API 키가 속한 클라우드 프로젝트와 연관된 파일 API에 업로드된 파일을 확인할 수 있게 해줍니다. 

In [18]:
import time

# 비디오 파일 처리 상태 확인
while video_file.state.name == "PROCESSING":
    # 처리 완료 대기 메시지 출력
    print("비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...")
    # 10초 대기
    time.sleep(10)
    # 비디오 파일 상태 갱신
    video_file = genai.get_file(video_file.name)

# 처리 실패 시 예외 발생
if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

# 처리 완료 메시지 출력
print(
    f"\n비디오 처리가 완료되었습니다!\n이제 대화를 시작할 수 있어요: " + video_file.uri
)

비디오 업로드 및 전처리가 완료될 때까지 잠시만 기다려주세요...

비디오 처리가 완료되었습니다!
이제 대화를 시작할 수 있어요: https://generativelanguage.googleapis.com/v1beta/files/lvj0w4tmclhv


비디오가 업로드된 후, `generate_content` 함수를 사용하여 Video 에 대한 질문을 요청할 수 있습니다.

In [20]:
# 프롬프트
prompt = "이 영상에 대해서 짧게 요약해 줄 수 있나요?"

# 모델을 Gemini 1.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# LLM 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}
)
# 결과 출력
print(response.text)

이 영상은 2024년 8월 패스트캠퍼스 월간 주주총회 라이브 편집본으로, '고급 문서 전처리 (LangGraph)'를 주제로 다룹니다.

**주요 내용:**

1.  **문제점 제기:** 기존 PDF 문서 파싱 방식은 텍스트만 긁어오기 때문에 문서 내 차트, 이미지, 표 등의 시각적 요소나 구조적 정보가 누락되거나 손상되어 RAG(Retrieval Augmented Generation) 시스템의 답변 품질이 떨어진다는 문제점을 지적합니다. 특히, 표 데이터는 단순 임베딩 시 질문과의 연관성이 낮아 검색에 잘 걸리지 않는 경우가 많습니다.

2.  **해결책 및 고급 전처리 파이프라인 소개:**
    *   **레이아웃 파서 활용:** Upstage Layout Analyzer와 같은 레이아웃 파서를 사용하여 PDF 문서를 텍스트, 이미지, 표 등 개별 요소(element)로 정확히 분리하고, 각 요소의 위치(바운딩 박스) 및 카테고리(제목, 본문, 이미지, 표 등) 정보를 JSON 형태로 추출하는 방법을 설명합니다.
    *   **이미지 처리:** 추출된 이미지의 바운딩 박스 정보를 활용하여 원본 PDF에서 해당 이미지를 정확히 잘라내어 별도의 PNG 파일로 저장합니다. 이는 나중에 이미지 검색 시 원본 이미지를 함께 제공하거나 멀티모달 임베딩을 구성할 때 유용합니다.
    *   **표 처리:** 표 데이터를 단순히 텍스트로 가져오는 것을 넘어, HTML 형태를 유지하거나 마크다운 형식으로 변환하여 표의 구조를 보존하고 LLM이 이해하기 쉽게 만듭니다. 또한, 표와 이미지에 대한 요약(Summary) 및 개체(Entity) 추출을 통해 텍스트 정보 외의 비정형 데이터를 풍부하게 만듭니다.
    *   **데이터 통합 및 활용:** 이렇게 분리, 처리, 요약된 모든 정보를 GraphState라는 단일 딕셔너리 객체에 저장하여 파일 경로, 페이지 번호, 메타데이터, 각 요소별 데이터(텍스트, 이미지 경로, 표 마크다운, 각 요소의 요약 등)를 통합 관리

아래는 스트림 출력 예제입니다. (`stream=True` 옵션 추가)


In [21]:
# 프롬프트 생성
prompt = "이 영상에서 Gencon 관련 언급한 부분의 시간을 알려주고, 어떤 내용을 말했는지 알려주세요."

# 모델을 Gemini 1.5 Flash로 설정
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# LLM 스트림 답변 요청
response = model.generate_content(
    [prompt, video_file], request_options={"timeout": 600}, stream=True
)

# 생성된 콘텐츠 출력
for chunk in response:
    print(chunk.text, end="", flush=True)

영상에서 Gencon 관련 언급된 부분은 **00:27부터 00:44**까지입니다.

주요 내용은 다음과 같습니다:

*   **젠콘 (Gencon) 이벤트 소개 (00:27)**: 2024년 9월 20일(금) 서울 코엑스에서 열리는 '2024 AI Conference for DEV'라는 이름의 행사입니다.
*   **이벤트 참여 및 혜택 (00:30)**: 이 행사는 무료로 진행되며, 라이브 방송 중간에 퀴즈가 출제되고 암호가 공개될 예정입니다. 이 암호를 설문지에 작성하면 경품을 받을 수 있다고 언급합니다.

## 파일 삭제

파일은 2일 후 자동으로 삭제되거나 `files.delete()`를 사용하여 수동으로 삭제할 수 있습니다.

In [9]:
# 파일 삭제
genai.delete_file(video_file.name)

# 삭제 완료 메시지 출력
print(f"영상을 삭제했습니다: {video_file.uri}")

영상을 삭제했습니다: https://generativelanguage.googleapis.com/v1beta/files/b1d0iktswld
